In [42]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [8]:
df = pd.read_csv('all_kindle_review.csv')

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [10]:
df = df[['rating' , 'reviewText']]

In [11]:
df.head()

,rating,reviewText
0,3,"Jace Rankin may be short, but he's nothing to ..."
1,5,Great short read. I didn't want to put it dow...
2,3,I'll start by saying this is the first of four...
3,3,Aggie is Angela Lansbury who carries pocketboo...
4,4,I did not expect this type of book to be in li...


In [12]:
df.tail()

,rating,reviewText
11995,4,Valentine cupid is a vampire- Jena and Ian ano...
11996,5,I have read all seven books in this series. Ap...
11997,3,This book really just wasn't my cuppa. The si...
11998,1,"tried to use it to charge my kindle, it didn't..."
11999,3,Taking Instruction is a look into the often hi...


In [15]:
df.shape

(12000, 2)

In [16]:
df.isnull().sum()

rating        0
reviewText    0
dtype: int64

In [19]:
df['rating'].unique

<bound method Series.unique of 0        3
1        5
2        3
3        3
4        4
        ..
11995    4
11996    5
11997    3
11998    1
11999    3
Name: rating, Length: 12000, dtype: int64>

In [20]:
df['rating'].value_counts()    

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [23]:
df['rating'] = df['rating'].apply(lambda x: 0 if x < 3 else 1)

In [24]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [25]:
df['reviewText'] = df['reviewText'].str.lower()

In [26]:
df.head()

,rating,reviewText
0,1,"jace rankin may be short, but he's nothing to ..."
1,1,great short read. i didn't want to put it dow...
2,1,i'll start by saying this is the first of four...
3,1,aggie is angela lansbury who carries pocketboo...
4,1,i did not expect this type of book to be in li...


In [ ]:
nltk.download('stopwords')

In [28]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split()])

In [31]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_text(x))

In [32]:
df.head()

,rating,reviewText
0,1,jace rankin may short he nothing mess man haul...
1,1,great short read didnt want put read one sitti...
2,1,ill start saying first four book wasnt expecti...
3,1,aggie angela lansbury carry pocketbook instead...
4,1,expect type book library pleased find price right


In [ ]:
X_train , X_test , y_train , y_test = train_test_split(df['reviewText'],df['rating'], test_size=0.2)

In [40]:
tfidf = TfidfVectorizer()


In [45]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [46]:
nb = GaussianNB().fit(X_train_tfidf, y_train)

In [47]:
y_pred = nb.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.5745833333333333
[[515 309]
 [712 864]]
              precision    recall  f1-score   support

           0       0.42      0.62      0.50       824
           1       0.74      0.55      0.63      1576

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.57      0.59      2400

